# FULL IPYNB — Baseline (Energy Reports 2024) vs Proposed Z-CATNet (Research-ready)

Notebook end-to-end:
1) Load dữ liệu (UCI txt hoặc CSV)  
2) Resample 15 phút → DLP 96 điểm/ngày  
3) Weekday pattern (mean/std) → Z-score (giống paper)  
4) Feature 99 = 96 Z + max/mean/min  
5) Nếu chưa có nhãn: pseudo-label 33 lớp (multi-label) để chạy demo end-to-end  
6) Train Baseline ER-ANN (y chang paper)  
7) Train Proposed Z-CATNet (CNN + BiLSTM + Self-Attention + Focal Loss)  
8) Evaluate + export CSV


## 0) Cài thư viện (nếu chạy local)

In [ ]:
# !pip install -q pandas numpy scikit-learn matplotlib tensorflow

## 1) Load dữ liệu (UCI txt hoặc CSV)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

DATA_TYPE = "UCI_TXT"   # "UCI_TXT" hoặc "CSV"

UCI_PATH = r"D:\AIL303c\data\household_power_consumption.txt"   # <-- SỬA

CSV_PATH = r"D:\your_data.csv"  # <-- SỬA
CSV_TS_COL = "timestamp"
CSV_VAL_COL = "power"
CSV_AGG = "mean"  # "mean"(kW) hoặc "sum"(kWh)

def load_series():
    if DATA_TYPE == "UCI_TXT":
        df = pd.read_csv(UCI_PATH, sep=";", na_values="?", low_memory=False)
        df["timestamp"] = pd.to_datetime(df["Date"] + " " + df["Time"], dayfirst=True, errors="coerce")
        df = df[["timestamp", "Global_active_power"]].dropna()
        df = df.sort_values("timestamp").set_index("timestamp")
        s = df["Global_active_power"].astype(float)
        return s, "kW_mean"
    else:
        df = pd.read_csv(CSV_PATH)
        df[CSV_TS_COL] = pd.to_datetime(df[CSV_TS_COL])
        df = df.sort_values(CSV_TS_COL).set_index(CSV_TS_COL)
        s = df[CSV_VAL_COL].astype(float)
        return s, ("kWh_sum" if CSV_AGG=="sum" else "kW_mean")

raw_series, series_kind = load_series()
print("Loaded:", raw_series.shape, "| kind:", series_kind)
raw_series.head()


## 2) Resample 15 phút + xử lý missing

In [ ]:
s_15m = raw_series.resample("15min").mean() if series_kind == "kW_mean" else raw_series.resample("15min").sum()
s_15m = s_15m.interpolate(method="time", limit_direction="both")
s_15m = s_15m.fillna(s_15m.median())
print("NaN after cleaning:", int(s_15m.isna().sum()))
s_15m.head(10)


## 3) DLP 96 điểm/ngày

In [ ]:
def build_daily_matrix(series_15m: pd.Series):
    days, idx = [], []
    for d, chunk in series_15m.groupby(series_15m.index.date):
        if len(chunk) < 96:
            continue
        arr = chunk.iloc[:96].values.astype(float)
        if np.isnan(arr).any():
            continue
        days.append(arr)
        idx.append(pd.Timestamp(d))
    X_raw = np.vstack(days) if len(days) else np.empty((0, 96))
    dates = pd.DatetimeIndex(idx)
    return X_raw, dates

X_raw, dates = build_daily_matrix(s_15m)
print("X_raw:", X_raw.shape, "| date range:", dates.min(), "->", dates.max())


## 4) Weekday pattern + Z-score (giống paper)

In [ ]:
def compute_weekday_patterns(X_raw, dates, min_days=5):
    patterns = {}
    for w in range(7):
        mask = (dates.weekday == w)
        Xw = X_raw[mask]
        if len(Xw) < min_days:
            continue
        mu = np.mean(Xw, axis=0)
        sd = np.std(Xw, axis=0, ddof=1)
        patterns[w] = (mu, sd)
    return patterns

def z_normalize_by_weekday(X_raw, dates, patterns, eps=1e-6):
    Z = np.zeros_like(X_raw, dtype=float)
    avail = sorted(patterns.keys())
    if not avail:
        raise ValueError("Không đủ dữ liệu để tạo weekday pattern. Cần nhiều ngày hơn.")
    for i, d in enumerate(dates):
        w = int(d.weekday())
        if w not in patterns:
            w = avail[0]
        mu, sd = patterns[w]
        sd_safe = np.where((sd < eps) | np.isnan(sd), 1.0, sd)
        Zi = (X_raw[i] - mu) / sd_safe
        Z[i] = np.nan_to_num(Zi, nan=0.0, posinf=0.0, neginf=0.0)
    return Z

patterns = compute_weekday_patterns(X_raw, dates, min_days=5)
Z = z_normalize_by_weekday(X_raw, dates, patterns)
print("pattern weekdays:", sorted(patterns.keys()))
print("Z:", Z.shape, "| NaN:", int(np.isnan(Z).sum()))


## 5) Feature 99 + plot 1 ngày

In [ ]:
daily_max = X_raw.max(axis=1)
daily_mean = X_raw.mean(axis=1)
daily_min = X_raw.min(axis=1)

X_99 = np.column_stack([Z, daily_max, daily_mean, daily_min])
print("X_99:", X_99.shape)

plt.figure(figsize=(10,4))
plt.plot(Z[0])
plt.title(f"Z-score DLP (date={dates[0].date()})")
plt.xlabel("slot (0..95)"); plt.ylabel("Z"); plt.grid(True, alpha=0.3)
plt.show()


## 6) Labels 33 lớp (pseudo-label nếu chưa có label thật)

In [ ]:
HAVE_TRUE_LABELS = False

SLOTS = {
    "night": np.arange(0, 24),
    "early": np.arange(24, 32),
    "work":  np.arange(32, 72),
    "eve":   np.arange(72, 96),
}

def catalog_33_pseudo(Z, X_raw, z_thr=2.0):
    n = Z.shape[0]
    Y = np.zeros((n, 33), dtype=int)
    out = (np.abs(Z) >= z_thr)
    night, early, work, eve = SLOTS["night"], SLOTS["early"], SLOTS["work"], SLOTS["eve"]
    nonwork = np.r_[night, early, eve]

    for i in range(n):
        out_any = out[i]
        meanZ, maxZ, minZ = np.mean(Z[i]), np.max(Z[i]), np.min(Z[i])
        f_all = np.mean(out_any)
        f_work = np.mean(out_any[work])
        f_non = np.mean(out_any[nonwork])

        if not out_any.any(): Y[i,0] = 1
        x = X_raw[i]
        if np.mean(np.isclose(np.diff(x), 0.0, atol=1e-4)) > 0.30: Y[i,1] = 1
        if f_work > 0.15: Y[i,2] = 1
        if out_any[work].any() and (not out_any[nonwork].any()): Y[i,3] = 1
        if out_any[work].any() and (not out_any[early].any()): Y[i,4] = 1
        if np.mean(out_any[night]) > 0.10 and np.mean(Z[i, night] > z_thr) > 0.05: Y[i,5] = 1
        if np.mean(out_any[early]) > 0.10: Y[i,6] = 1
        if (not out_any[night].any()) and out_any[work].any(): Y[i,7] = 1
        if out_any[night].any(): Y[i,8] = 1
        if np.mean(out_any[early]) > 0.25: Y[i,9] = 1
        if np.mean(out_any[night]) > 0.25: Y[i,10] = 1
        if (f_non > f_work + 0.05) and (abs(meanZ) < 0.2): Y[i,11] = 1; Y[i,12] = 1
        if (0 < f_all < 0.05) and (abs(meanZ) < 0.2): Y[i,13] = 1
        if (f_work > f_non + 0.05) and (abs(meanZ) < 0.2): Y[i,14] = 1
        if meanZ > 0.5: Y[i,15] = 1
        if meanZ < -0.5: Y[i,16] = 1
        if (np.max(Z[i, work]) >= z_thr) and (np.max(Z[i, eve]) >= z_thr): Y[i,17] = 1
        if (np.mean(Z[i, work]) < -0.3) and (np.max(Z[i, work]) < 0.5): Y[i,18] = 1
        if (minZ > -0.2) and (np.mean(Z[i] > 0.5) > 0.10): Y[i,19] = 1
        if minZ <= -z_thr: Y[i,20] = 1
        if meanZ > 0.7: Y[i,21] = 1
        if meanZ < -0.7: Y[i,22] = 1
        if maxZ >= z_thr: Y[i,23] = 1
        if (np.max(Z[i]) < 0.3) and (np.mean(Z[i, work]) < -0.4): Y[i,24] = 1
        if minZ >= z_thr: Y[i,25] = 1
        if minZ <= -z_thr: Y[i,26] = 1
        if np.mean(Z[i, work]) > 0.6: Y[i,27] = 1
        if (np.min(Z[i]) > -0.2) and (np.mean(Z[i] > 0.4) > 0.20): Y[i,28] = 1
        if np.max(Z[i, work]) > (z_thr + 0.5): Y[i,29] = 1
        if np.mean(Z[i, work]) < -0.6: Y[i,30] = 1
        if Y[i,25] == 1: Y[i,31] = 1
        if Y[i,26] == 1: Y[i,32] = 1
    return Y

if HAVE_TRUE_LABELS:
    raise NotImplementedError("Load nhãn thật của bạn vào Y_33 ở đây.")
else:
    Y_33 = catalog_33_pseudo(Z, X_raw, z_thr=2.0)

print("Y_33:", Y_33.shape, "| avg labels/day:", float(Y_33.sum(axis=1).mean()))


## 7) Split train/test + Standardize

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X_99, Y_33, test_size=0.2, random_state=42, shuffle=True)

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s  = scaler.transform(X_test)

print("Train:", X_train_s.shape, y_train.shape, "| Test:", X_test_s.shape, y_test.shape)


## 8) Baseline ER-ANN (paper)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def build_er_ann():
    model = keras.Sequential([
        layers.Input(shape=(99,)),
        layers.Dense(100, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(33, activation="sigmoid")
    ])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
                  loss="binary_crossentropy",
                  metrics=[keras.metrics.BinaryAccuracy(name="bin_acc")])
    return model

callbacks = [
    keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(patience=5)
]

er_ann = build_er_ann()
hist_er = er_ann.fit(X_train_s, y_train, validation_split=0.2, epochs=50, batch_size=64, callbacks=callbacks, verbose=1)


## 9) Proposed Z-CATNet (CNN + BiLSTM + Self-Attention + Focal Loss)

In [ ]:
def focal_loss(gamma=2.0, alpha=0.25):
    def loss(y_true, y_pred):
        eps = 1e-7
        y_pred = tf.clip_by_value(y_pred, eps, 1. - eps)
        ce = -y_true * tf.math.log(y_pred) - (1 - y_true) * tf.math.log(1 - y_pred)
        w = alpha * y_true * tf.pow(1 - y_pred, gamma) + (1 - alpha) * (1 - y_true) * tf.pow(y_pred, gamma)
        return tf.reduce_mean(w * ce)
    return loss

class SelfAttention(layers.Layer):
    def __init__(self, d_k=64, **kwargs):
        super().__init__(**kwargs)
        self.d_k = d_k
        self.q = layers.Dense(d_k)
        self.k = layers.Dense(d_k)
        self.v = layers.Dense(d_k)

    def call(self, x):
        Q = self.q(x); K = self.k(x); V = self.v(x)
        score = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(tf.cast(self.d_k, tf.float32))
        attn = tf.nn.softmax(score, axis=-1)
        return tf.matmul(attn, V)

def split_inputs(X99_scaled):
    z = X99_scaled[:, :96].astype(np.float32)[..., None]   # (n,96,1)
    stats = X99_scaled[:, 96:].astype(np.float32)          # (n,3)
    return z, stats

z_train, s_train = split_inputs(X_train_s)
z_test,  s_test  = split_inputs(X_test_s)
print("z_train:", z_train.shape, "| s_train:", s_train.shape)

def build_z_catnet(num_classes=33):
    inp_z = layers.Input(shape=(96,1), name="z_input")
    x = layers.Conv1D(32, 5, padding="same", activation="relu")(inp_z)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(64, 5, padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
    x = SelfAttention(d_k=64)(x)
    x = layers.GlobalAveragePooling1D()(x)

    inp_s = layers.Input(shape=(3,), name="stats_input")
    s = layers.Dense(16, activation="relu")(inp_s)
    s = layers.BatchNormalization()(s)
    s = layers.Dense(8, activation="relu")(s)

    h = layers.Concatenate()([x, s])
    h = layers.Dense(128, activation="relu")(h)
    h = layers.BatchNormalization()(h)
    h = layers.Dropout(0.4)(h)
    h = layers.Dense(64, activation="relu")(h)
    h = layers.Dropout(0.3)(h)

    out = layers.Dense(num_classes, activation="sigmoid")(h)

    model = keras.Model(inputs=[inp_z, inp_s], outputs=out, name="Z_CATNet")
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                  loss=focal_loss(gamma=2.0, alpha=0.25),
                  metrics=[keras.metrics.BinaryAccuracy(name="bin_acc")])
    return model

z_catnet = build_z_catnet()
hist_cat = z_catnet.fit({"z_input":z_train,"stats_input":s_train},
                        y_train,
                        validation_split=0.2, epochs=50, batch_size=64,
                        callbacks=callbacks, verbose=1)


## 10) Evaluation (micro precision + per-class Precision/FPR/FNR)

In [ ]:
from sklearn.metrics import precision_score

def per_class_metrics(y_true, y_hat):
    rows = []
    for j in range(y_true.shape[1]):
        yt, yp = y_true[:, j], y_hat[:, j]
        TP = np.sum((yt==1) & (yp==1))
        FP = np.sum((yt==0) & (yp==1))
        FN = np.sum((yt==1) & (yp==0))
        TN = np.sum((yt==0) & (yp==0))
        precision = TP / (TP + FP + 1e-9)
        fpr = FP / (FP + TN + 1e-9)
        fnr = FN / (FN + TP + 1e-9)
        acc = (TP + TN) / (TP + TN + FP + FN + 1e-9)
        rows.append([j+1, acc, precision, fpr, fnr, int(yt.sum())])
    return pd.DataFrame(rows, columns=["class","accuracy","precision","FPR","FNR","support"])

prob_er = er_ann.predict(X_test_s, verbose=0)
pred_er = (prob_er >= 0.5).astype(int)

prob_cat = z_catnet.predict({"z_input":z_test,"stats_input":s_test}, verbose=0)
pred_cat = (prob_cat >= 0.5).astype(int)

micro_er = precision_score(y_test.reshape(-1), pred_er.reshape(-1), zero_division=0)
micro_cat = precision_score(y_test.reshape(-1), pred_cat.reshape(-1), zero_division=0)

print("Micro Precision ER-ANN  :", round(float(micro_er), 4))
print("Micro Precision Z-CATNet:", round(float(micro_cat), 4))

m_er = per_class_metrics(y_test, pred_er).sort_values("support", ascending=False)
m_cat = per_class_metrics(y_test, pred_cat).sort_values("support", ascending=False)

display(m_er.head(12))
display(m_cat.head(12))


## 11) Plot learning curves

In [ ]:
plt.figure()
plt.plot(hist_er.history["bin_acc"], label="ER train bin_acc")
plt.plot(hist_er.history["val_bin_acc"], label="ER val bin_acc")
plt.xlabel("epoch"); plt.ylabel("bin_acc"); plt.legend(); plt.grid(True, alpha=0.3)
plt.show()

plt.figure()
plt.plot(hist_cat.history["bin_acc"], label="Z-CATNet train bin_acc")
plt.plot(hist_cat.history["val_bin_acc"], label="Z-CATNet val bin_acc")
plt.xlabel("epoch"); plt.ylabel("bin_acc"); plt.legend(); plt.grid(True, alpha=0.3)
plt.show()


## 12) Export CSV (đưa vào paper)

In [ ]:
summary = pd.DataFrame({
    "model": ["ER-ANN (paper)", "Z-CATNet (proposed)"],
    "micro_precision": [micro_er, micro_cat],
})
display(summary)

m_er.to_csv("metrics_er_ann.csv", index=False)
m_cat.to_csv("metrics_z_catnet.csv", index=False)
summary.to_csv("summary_metrics.csv", index=False)

print("Saved: metrics_er_ann.csv, metrics_z_catnet.csv, summary_metrics.csv")
